In [1]:
import os
#os.system('pwd')

import torch
import torchhd
from torchhd.models import Centroid
from torchhd import embeddings
import torchvision
from torchvision.datasets import MNIST
from torchhd.datasets.isolet import ISOLET
from torchhd.datasets import EMGHandGestures
#from torch_geometric.datasets import TUDataset
import torchmetrics
from tqdm import tqdm
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import random

import matplotlib.pyplot as plt
import pandas as pd

from torch.utils.data import DataLoader, TensorDataset

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using {} device".format(device))

BATCH_SIZE = 64

Using cuda device


In [3]:
class Encoder_rand(nn.Module):
    def __init__(self, out_features, size, levels):
        super(Encoder_rand, self).__init__()
        #self.flatten = torch.nn.Flatten(start_dim=-2)
        self.project = embeddings.Sinusoid(size, out_features)
        self.name="RandomProjectionEncoder"
    
    def forward(self, x):
        #x = self.flatten(x)
        sample_hv = self.project(x)
        #sample_hv = torchhd.multiset(sample_hv)
        return torchhd.hard_quantize(sample_hv)

###### =====================================BASE LEVEL=============================================

class Encoder_base(nn.Module):
    def __init__(self, out_features, size, levels):
        super(Encoder_base, self).__init__()
        #self.flatten = torch.nn.Flatten(start_dim=-2)
        self.position = embeddings.Random(size, out_features)
        self.value = embeddings.Level(levels, out_features)
        self.name="BaseLevelEncoder"

    def forward(self, x):
        #x = self.flatten(x)
        sample_hv = torchhd.bind(self.position.weight, self.value(x))
        sample_hv = torchhd.multiset(sample_hv)
        return torchhd.hard_quantize(sample_hv)


def inject_fault(classHyperVectors, bits, amount):
    rng = np.random.default_rng()
    fault_index = rng.integers(0, classHyperVectors.weight.data.shape[0] * classHyperVectors.weight.data.shape[1] * bits, amount)
    flattened = classHyperVectors.weight.data.flatten().clone()
    if bits == 1:
        flattened = F.relu(flattened).type(torch.int)
    else:
        flattened = flattened.type(torch.int)
    #print(flattened[:100])
    bitstring = np.array(list(''.join([format(weight & (2**bits-1),'0'+str(bits)+'b') for weight in flattened]))).astype(int)
    bitstring[fault_index] = (bitstring[fault_index]-1) * -1
    bitstring = bitstring.astype(str)
    bitstring = [twos_comp(int("".join(bitstring[i:i + bits]), 2),bits) for i in range(0, len(bitstring), bits)]
    #print(bitstring[:100])
    bitstring = torch.tensor(bitstring).reshape(classHyperVectors.weight.data.shape[0], classHyperVectors.weight.data.shape[1])
    if bits == 1:
        #bits are reversed, -1 needs to be 1 and 0 needs to become -1. This happens because we make -1 to 0 with F.relu()!
        bitstring *= -1
        bitstring *= 2
        bitstring -= 1
    return bitstring

def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val                         # return positive value as is

def train_model(encoder, levels, d, size, num_classes, q, dataset, train_ld, test_ld, flatten = False, runs=10):
    #train 10 times and take the best model (and encoder)
    best_accuracy = 0
    best_model = Centroid(d, num_classes)
    if encoder == "RandomProjectionEncoder":
        best_encoder = Encoder_rand(d, size, levels).to(device)
    elif encoder =="BaseLevelEncoder":
        best_encoder = Encoder_base(d, size, levels).to(device)

    for i in range(runs):
        if encoder == "RandomProjectionEncoder":
            encode = Encoder_rand(d, size, levels).to(device)
        elif encoder =="BaseLevelEncoder":
            encode = Encoder_base(d, size, levels).to(device)
            
        model = Centroid(d, num_classes)
        model = model.to(device)

        with torch.no_grad():
            for samples, labels in tqdm(train_ld, desc="Training"):
                samples = samples.to(device)
                if flatten:
                    samples = samples.flatten(start_dim=-2)
                    samples = samples.reshape(samples.shape[0], samples.shape[-1])
                labels = labels.to(device)
                samples_hv = encode(samples)
                model.add(samples_hv, labels)
        
        accuracy = torchmetrics.Accuracy("multiclass", num_classes=num_classes)
        dotsim = []
        dotsim10 = []
        classification = []
        lb = []
        with torch.no_grad():
            model.min_max_normalize(quantize = q)

            for samples, labels in tqdm(test_ld, desc="Testing"):
                samples = samples.to(device)
                if flatten:
                    samples = samples.flatten(start_dim=-2)
                    samples = samples.reshape(samples.shape[0], samples.shape[-1])
                samples_hv = encode(samples)
                outputs = model(samples_hv, dot=True)
                accuracy.update(outputs.cpu(), labels)
                dotsim.extend(torch.max(outputs.cpu(),dim=1).values.tolist())
                dotsim10.extend(np.sort(outputs.cpu().detach().numpy()))
                classification.extend(torch.argmax(outputs.cpu(),dim=1).tolist())
                lb.extend(labels.tolist())
            dotsimbase = np.array(dotsim)
            dotsimbase10 = np.array(dotsim10)
            lb = np.array(lb)
            classification = np.array(classification)
    
        #only keep the best model
        current_accuracy = accuracy.compute().item() * 100
        if current_accuracy > best_accuracy:
            best_model.weight.data = model.weight.data.detach().clone()
            best_encoder = encode
            np.save("./models/"+ dataset +"/" + encode.name + "/normal/" + "quantize_"+str(q)+"_"+str(d)+"_"+'dotsimbase10'+'.npy', dotsimbase10)
            np.save("./models/"+ dataset +"/" + encode.name + "/normal/" + "quantize_"+str(q)+"_"+str(d)+"_"+'dotsimbase'+'.npy', dotsimbase)
            np.save("./models/"+ dataset +"/" + encode.name + "/normal/" + "quantize_"+str(q)+"_"+str(d)+"_"+'classified'+'.npy', classification)
            np.save("./models/"+ dataset +"/" + encode.name + "/normal/" + "quantize_"+str(q)+"_"+str(d)+"_"+'labels'+'.npy', lb)
            torch.save(model.weight, "./models/"+ dataset + "/" + encode.name + "/" + "quantize_" + str(q) + "_"+str(d) + ".pt")
            torch.save(encode, "./models/"+ dataset + "/" + encode.name + "/" + "enc_" + "quantize_" + str(q) + "_"+str(d) + ".pt")
    return best_model, best_encoder, accuracy

In [4]:
DIMENSION = [1000, 5000, 10000]

quantize = [1, 8]

encoders = ["RandomProjectionEncoder", "BaseLevelEncoder"]

averaging = [0,10]

In [5]:
#ISOLET

# DIMENSIONS = 5000  # number of hypervector dimensions
NUM_LEVELS = 100


train_ds = ISOLET("data/", train=True, download=False)
train_ld = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)

test_ds = ISOLET("data/", train=False, download=False)
test_ld = torch.utils.data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

num_classes = len(train_ds.classes)

for d in DIMENSION:
    for q in quantize:
        for Encoder in encoders:
            
            model, encode, accuracy = train_model(Encoder,
                                        NUM_LEVELS,
                                        d,
                                        train_ds[0][0].size(-1),
                                        num_classes,
                                        q,
                                        "isolet",
                                        train_ld,
                                        test_ld)

            #inject fault
            print("quantize_"+str(q)+"_"+str(d))
            print(f"Testing accuracy of {(accuracy.compute().item() * 100):.3f}%")
            w = model.weight.data.detach().clone()
            
            for avg in range(averaging[0], averaging[1]):
                df = pd.DataFrame(columns=[
                'Fault %', 'Accuracy'
                ])
                faultyPoints = 0

                while faultyPoints < d*num_classes*q:
                    
                    accuracies = []
                    dotsim = []
                    dotsim10 = []
                    classification = []
                    model_faulty = Centroid(d, num_classes)
                    model_faulty = model_faulty.to(device)
                    model_faulty.weight.data = w.clone()
                    
                    model_faulty.weight.data = inject_fault(model_faulty, amount=faultyPoints, bits=q).to(device)

                    accuracy = torchmetrics.Accuracy("multiclass", num_classes=num_classes)
                    with torch.no_grad():
                        # model.normalize(quantize = q)
                        for samples, labels in tqdm(test_ld, desc="Testing", disable= True):
                            samples = samples.to(device)
                            samples_hv = encode(samples)
                            outputs = model_faulty(samples_hv, dot=True)
                            accuracy.update(outputs.cpu(), labels)
                            dotsim.extend(torch.max(outputs.cpu(),dim=1).values.tolist())
                            dotsim10.extend(np.sort(outputs.cpu().detach().numpy()))
                            classification.extend(torch.argmax(outputs.cpu(),dim=1).tolist())
                    
                    dotsimbase = np.array(dotsim)
                    dotsimbase10 = np.array(dotsim10) 
                    classification = np.array(classification)

                    np.save("./models/isolet/" + encode.name + "/faulty/" + "quantize_"+str(q)+"_"+str(d)+"_"+str(avg)+"_"+"dotsimbase10"+".npy", dotsimbase10)
                    np.save("./models/isolet/" + encode.name + "/faulty/" + "quantize_"+str(q)+"_"+str(d)+"_"+str(avg)+"_"+"dotsimbase"+".npy", dotsimbase)
                    np.save("./models/isolet/" + encode.name + "/faulty/" + "quantize_"+str(q)+"_"+str(d)+"_"+str(avg)+"_"+"classified"+".npy", classification)

                    row = {
                            'Fault %': [faultyPoints/(d*num_classes*q)],
                            'Accuracy': [accuracy.compute().item() * 100],
                        }
                    faultyPoints = int(faultyPoints + (d*num_classes*q)/100)
                    row = pd.DataFrame.from_dict(row)
                    df = pd.concat([df,row], ignore_index=True)
                print(df)
                df.to_excel("./models/isolet/" + encode.name + "/faulty/quantize_"+str(q)+"_"+str(d)+"_"+str(avg)+"result.xlsx", index=False)               
        

Testing: 100%|██████████| 25/25 [00:00<00:00, 579.95it/s]


quantize_1_1000
Testing accuracy of 80.436%
    Fault %   Accuracy
0      0.00  80.436176
1      0.01  79.602307
2      0.02  79.602307
3      0.03  77.421427
4      0.04  76.844132
..      ...        ...
95     0.95   0.128287
96     0.96   0.384862
97     0.97   0.000000
98     0.98   0.064144
99     0.99   0.449006

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  80.436176
1      0.01  79.281592
2      0.02  79.602307
3      0.03  79.089159
4      0.04  76.010263
..      ...        ...
95     0.95   0.000000
96     0.96   0.384862
97     0.97   0.000000
98     0.98   0.064144
99     0.99   0.064144

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  80.436176
1      0.01  80.051315
2      0.02  78.062862
3      0.03  78.640151
4      0.04  76.779985
..      ...        ...
95     0.95   0.192431
96     0.96   0.064144
97     0.97   0.064144
98     0.98   0.192431
99     0.99   0.064144

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  80.436176
1      0.0

Testing: 100%|██████████| 25/25 [00:00<00:00, 372.96it/s]


quantize_1_1000
Testing accuracy of 68.185%
    Fault %   Accuracy
0      0.00  68.184733
1      0.01  64.400256
2      0.02  63.758820
3      0.03  58.114177
4      0.04  56.703013
..      ...        ...
95     0.95   0.513149
96     0.96   0.513149
97     0.97   0.898012
98     0.98   0.513149
99     0.99   1.282874

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  68.184733
1      0.01  63.887107
2      0.02  63.181525
3      0.03  59.781909
4      0.04  60.102630
..      ...        ...
95     0.95   0.962155
96     0.96   0.192431
97     0.97   0.449006
98     0.98   0.320718
99     0.99   0.513149

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  68.184733
1      0.01  64.528543
2      0.02  61.449647
3      0.03  61.321360
4      0.04  57.280308
..      ...        ...
95     0.95   0.769724
96     0.96   0.320718
97     0.97   0.577293
98     0.98   0.256575
99     0.99   0.384862

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  68.184733
1      0.0

Testing: 100%|██████████| 25/25 [00:00<00:00, 411.55it/s]


...

quantize_1_10000
Testing accuracy of 82.745%
    Fault %   Accuracy
0      0.00  82.745349
1      0.01  82.488775
2      0.02  80.756897
3      0.03  81.270045
4      0.04  80.051315
..      ...        ...
95     0.95   0.000000
96     0.96   0.064144
97     0.97   0.000000
98     0.98   0.064144
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  82.745349
1      0.01  82.103914
2      0.02  81.590766
3      0.03  80.307889
4      0.04  79.730594
..      ...        ...
95     0.95   0.064144
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  82.745349
1      0.01  81.847340
2      0.02  81.526619
3      0.03  80.692750
4      0.04  79.666454
..      ...        ...
95     0.95   0.128287
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.128287

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  82.745349
1      0.

Testing: 100%|██████████| 25/25 [00:00<00:00, 451.95it/s]


quantize_8_10000
Testing accuracy of 88.582%
    Fault %   Accuracy
0      0.00  88.582426
1      0.01  88.454139
2      0.02  88.646567
3      0.03  87.235409
4      0.04  86.593968
..      ...        ...
95     0.95   0.000000
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  88.582426
1      0.01  88.389993
2      0.02  88.325852
3      0.03  88.005131
4      0.04  87.556124
..      ...        ...
95     0.95   0.000000
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  88.582426
1      0.01  88.903141
2      0.02  88.133419
3      0.03  87.427837
4      0.04  87.171263
..      ...        ...
95     0.95   0.000000
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  88.582426
1      0.

Testing: 100%|██████████| 25/25 [00:00<00:00, 66.24it/s]


quantize_8_10000
Testing accuracy of 83.579%
    Fault %   Accuracy
0      0.00  83.579218
1      0.01  82.296342
2      0.02  82.360488
3      0.03  78.704298
4      0.04  78.383577
..      ...        ...
95     0.95   0.000000
96     0.96   0.000000
97     0.97   0.128287
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  83.579218
1      0.01  83.130211
2      0.02  82.360488
3      0.03  79.666454
4      0.04  80.051315
..      ...        ...
95     0.95   0.000000
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  83.579218
1      0.01  81.590766
2      0.02  79.987168
3      0.03  82.488775
4      0.04  81.975627
..      ...        ...
95     0.95   0.000000
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  83.579218
1      0.

In [6]:
#UCI_HAR_Dataset

# Paths to the dataset
train_data_path = 'data/UCI_HAR_Dataset/train/X_train.txt'
train_labels_path = 'data/UCI_HAR_Dataset/train/y_train.txt'
test_data_path = 'data/UCI_HAR_Dataset/test/X_test.txt'
test_labels_path = 'data/UCI_HAR_Dataset/test/y_test.txt'

# Load the dataset
train_data = pd.read_csv(train_data_path, delim_whitespace=True, header=None)
train_labels = pd.read_csv(train_labels_path, delim_whitespace=True, header=None)
test_data = pd.read_csv(test_data_path, delim_whitespace=True, header=None)
test_labels = pd.read_csv(test_labels_path, delim_whitespace=True, header=None)

# Convert data to PyTorch tensors
train_data_tensor = torch.tensor(train_data.values, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels.values.squeeze(), dtype=torch.long) - 1
test_data_tensor = torch.tensor(test_data.values, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels.values.squeeze(), dtype=torch.long) - 1

# Create TensorDatasets
train_dataset = TensorDataset(train_data_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_data_tensor, test_labels_tensor)

# Create DataLoaders
train_ld = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_ld = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

NUM_LEVELS = 1000
IMG_SIZE = 561
num_classes = 7 #len(train_loader.classes)
print(num_classes)

for d in DIMENSION:
    for q in quantize:
        for Encoder in encoders:

            model, encode, accuracy = train_model(Encoder,
                                        NUM_LEVELS,
                                        d,
                                        IMG_SIZE,
                                        num_classes,
                                        q,
                                        "ucihar",
                                        train_ld,
                                        test_ld)
            
            print("quantize_"+str(q)+"_"+str(d))
            print(f"Testing accuracy of {(accuracy.compute().item() * 100):.3f}%")

            w = model.weight.data.detach().clone()
            for avg in range(averaging[0], averaging[1]):
                df = pd.DataFrame(columns=[
                'Fault %', 'Accuracy'
                ])
                faultyPoints = 0
                
                while faultyPoints < d*num_classes*q:
                    
                    accuracies = []
                    dotsim = []
                    dotsim10 = []
                    classification = []
                    model_faulty = Centroid(d, num_classes)
                    model_faulty = model_faulty.to(device)
                    model_faulty.weight.data = w.clone()
                    
                    model_faulty.weight.data = inject_fault(model_faulty, amount=faultyPoints, bits=q).to(device)

                    accuracy = torchmetrics.Accuracy("multiclass", num_classes=num_classes)
                    with torch.no_grad():
                        # model.normalize(quantize = q)
                        for samples, labels in tqdm(test_ld, desc="Testing", disable= True):
                            samples = samples.to(device)
                            samples_hv = encode(samples)
                            outputs = model_faulty(samples_hv, dot=True)
                            accuracy.update(outputs.cpu(), labels)
                            dotsim.extend(torch.max(outputs.cpu(),dim=1).values.tolist())
                            dotsim10.extend(np.sort(outputs.cpu().detach().numpy()))
                            classification.extend(torch.argmax(outputs.cpu(),dim=1).tolist())
                    
                    dotsimbase = np.array(dotsim)
                    dotsimbase10 = np.array(dotsim10) 
                    classification = np.array(classification)

                    np.save("./models/ucihar/" + encode.name + "/faulty/" + "quantize_"+str(q)+"_"+str(d)+"_"+str(avg)+"_"+"dotsimbase10"+".npy", dotsimbase10)
                    np.save("./models/ucihar/" + encode.name + "/faulty/" + "quantize_"+str(q)+"_"+str(d)+"_"+str(avg)+"_"+"dotsimbase"+".npy", dotsimbase)
                    np.save("./models/ucihar/" + encode.name + "/faulty/" + "quantize_"+str(q)+"_"+str(d)+"_"+str(avg)+"_"+"classified"+".npy", classification)

                    row = {
                            'Fault %': [faultyPoints/(d*num_classes*q)],
                            'Accuracy': [accuracy.compute().item() * 100],
                        }
                    faultyPoints = int(faultyPoints + (d*num_classes*q)/100)
                    row = pd.DataFrame.from_dict(row)
                    df = pd.concat([df,row], ignore_index=True)
                print(df)
                df.to_excel("./models/ucihar/" + encode.name + "/faulty/quantize_"+str(q)+"_"+str(d)+"_"+str(avg)+"result.xlsx", index=False)               
        

7


Testing: 100%|██████████| 47/47 [00:00<00:00, 569.09it/s]


quantize_1_1000
Testing accuracy of 79.980%
    Fault %   Accuracy
0      0.00  79.979640
1      0.01  79.029524
2      0.02  77.909738
3      0.03  75.907701
4      0.04  78.791994
..      ...        ...
95     0.95   0.000000
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]


    Fault %   Accuracy
0      0.00  79.979640
1      0.01  76.382762
2      0.02  75.059384
3      0.03  76.518494
4      0.04  72.412622
..      ...        ...
95     0.95   0.000000
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  79.979640
1      0.01  77.807939
2      0.02  75.398713
3      0.03  78.486598
4      0.04  69.019341
..      ...        ...
95     0.95   0.000000
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  79.979640
1      0.01  80.861896
2      0.02  73.905665
3      0.03  76.993555
4      0.04  72.853750
..      ...        ...
95     0.95   0.000000
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  79.979640
1      0.01  78.147268
2      0.02  77.400744
3      0

Testing: 100%|██████████| 47/47 [00:00<00:00, 346.98it/s]


...

quantize_1_10000
Testing accuracy of 82.898%
    Fault %   Accuracy
0      0.00  82.897860
1      0.01  80.318969
2      0.02  79.945707
3      0.03  80.454701
4      0.04  76.620293
..      ...        ...
95     0.95   0.000000
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  82.897860
1      0.01  78.520530
2      0.02  82.354939
3      0.03  79.165250
4      0.04  79.708177
..      ...        ...
95     0.95   0.000000
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  82.897860
1      0.01  81.744146
2      0.02  77.977604
3      0.03  79.640311
4      0.04  75.059384
..      ...        ...
95     0.95   0.000000
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  82.897860
1      0.

Testing: 100%|██████████| 47/47 [00:00<00:00, 657.55it/s]


quantize_8_10000
Testing accuracy of 85.646%
    Fault %   Accuracy
0      0.00  85.646421
1      0.01  85.951817
2      0.02  85.850018
3      0.03  83.678317
4      0.04  86.223280
..      ...        ...
95     0.95   0.000000
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  85.646421
1      0.01  85.307091
2      0.02  85.680354
3      0.03  84.764165
4      0.04  85.408890
..      ...        ...
95     0.95   0.000000
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  85.646421
1      0.01  85.951817
2      0.02  85.408890
3      0.03  85.273159
4      0.04  84.662366
..      ...        ...
95     0.95   0.000000
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  85.646421
1      0.

Testing: 100%|██████████| 47/47 [00:00<00:00, 72.24it/s]


quantize_8_10000
Testing accuracy of 82.016%
    Fault %   Accuracy
0      0.00  82.015610
1      0.01  83.406854
2      0.02  83.440787
3      0.03  71.903628
4      0.04  77.841872
..      ...        ...
95     0.95   0.000000
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  82.015610
1      0.01  81.778079
2      0.02  80.285037
3      0.03  83.440787
4      0.04  68.713945
..      ...        ...
95     0.95   0.000000
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  82.015610
1      0.01  80.692232
2      0.02  69.121140
3      0.03  83.101457
4      0.04  81.099421
..      ...        ...
95     0.95   0.000000
96     0.96   0.000000
97     0.97   0.000000
98     0.98   0.000000
99     0.99   0.000000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  82.015610
1      0.

In [7]:
#MNIST dataset

transform = torchvision.transforms.ToTensor()


train_ds = MNIST("data", train=True, transform=transform, download=True)
train_ld = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_ds = MNIST("data", train=False, transform=transform, download=True)
test_ld = torch.utils.data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)



NUM_LEVELS = 256
IMG_SIZE = 28*28
num_classes = 10 

for d in DIMENSION:
    for q in quantize:
        for enc in encoders:
            print("quantize_"+str(q)+"_"+str(d))
            model, encode, accuracy = train_model(enc,
                                        NUM_LEVELS,
                                        d,
                                        IMG_SIZE,
                                        num_classes,
                                        q,
                                        "mnist",
                                        train_ld,
                                        test_ld,
                                        flatten=True,runs=10)
            
            print(f"Testing accuracy of {(accuracy.compute().item() * 100):.3f}%")

            w = model.weight.data.detach().clone()
            #inject fault
            for avg in range(averaging[0], averaging[1]):
                faultyPoints = 0
                w = model.weight.data.detach().clone()
                df = pd.DataFrame(columns=[
                        'Fault %', 'Accuracy'
                    ])
                while faultyPoints < d*num_classes*q:

                    accuracies = []
                    dotsim = []
                    dotsim10 = []
                    classification = []
                    model_faulty = Centroid(d, num_classes)
                    model_faulty = model_faulty.to(device)
                    model_faulty.weight.data = w.clone()
                    model_faulty.weight.data = inject_fault(model_faulty, amount=faultyPoints, bits=q).to(device)
                    
                    accuracy = torchmetrics.Accuracy("multiclass", num_classes=num_classes)
                    with torch.no_grad():
                        # model.normalize(quantize = q)
                        for samples, labels in tqdm(test_ld, desc="Testing", disable= True):
                            samples = samples.to(device)
                            samples = samples.flatten(start_dim=-2)
                            samples_hv = encode(samples)
                            outputs = model_faulty(samples_hv, dot=True)
                            accuracy.update(torch.argmax(outputs.cpu(),dim=-1).flatten(), labels)
                            dotsim.extend(torch.max(outputs.cpu(),dim=1).values.tolist())
                            dotsim10.extend(np.sort(outputs.cpu().detach().numpy()))
                            classification.extend(torch.argmax(outputs.cpu(),dim=1).tolist())
                    
                    dotsimbase = np.array(dotsim)
                    dotsimbase10 = np.array(dotsim10)
                    classification = np.array(classification)

                    np.save("./models/mnist/" + encode.name + "/faulty/" + "quantize_"+str(q)+"_"+str(d)+"_"+str(avg)+"_"+"dotsimbase10"+".npy", dotsimbase10)
                    np.save("./models/mnist/" + encode.name + "/faulty/" + "quantize_"+str(q)+"_"+str(d)+"_"+str(avg)+"_"+"dotsimbase"+".npy", dotsimbase)
                    np.save("./models/mnist/" + encode.name + "/faulty/" + "quantize_"+str(q)+"_"+str(d)+"_"+str(avg)+"_"+"classified"+".npy", classification)

                    row = {
                            'Fault %': [faultyPoints/(d*num_classes*q)],
                            'Accuracy': [accuracy.compute().item() * 100],
                        }
                    faultyPoints = int(faultyPoints + (d*num_classes*q)/100)
                    row = pd.DataFrame.from_dict(row)
                    df = pd.concat([df,row], ignore_index=True)
                print(df)
                df.to_excel("./models/mnist/" + encode.name + "/faulty/quantize_"+str(q)+"_"+str(d)+"_"+str(avg)+"result.xlsx", index=False)               
        

quantize_1_1000


Testing: 100%|██████████| 157/157 [00:00<00:00, 191.81it/s]


Testing accuracy of 78.400%


    Fault %   Accuracy
0      0.00  78.399998
1      0.01  78.219998
2      0.02  77.969998
3      0.03  76.779997
4      0.04  75.050002
..      ...        ...
95     0.95   2.080000
96     0.96   2.040000
97     0.97   1.440000
98     0.98   2.780000
99     0.99   3.620000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  78.399998
1      0.01  77.969998
2      0.02  76.700002
3      0.03  76.870000
4      0.04  76.480001
..      ...        ...
95     0.95   2.800000
96     0.96   0.990000
97     0.97   2.240000
98     0.98   1.780000
99     0.99   1.090000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  78.399998
1      0.01  76.940000
2      0.02  77.770001
3      0.03  75.400001
4      0.04  75.180000
..      ...        ...
95     0.95   0.960000
96     0.96   1.740000
97     0.97   2.850000
98     0.98   1.010000
99     0.99   1.180000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  78.399998
1      0.01  77.829999
2      0.02  78.270000
3      0

Testing: 100%|██████████| 157/157 [00:01<00:00, 142.27it/s]


...

Testing accuracy of 80.790%
    Fault %   Accuracy
0      0.00  80.790001
1      0.01  80.250001
2      0.02  80.180001
3      0.03  80.119997
4      0.04  79.079998
..      ...        ...
95     0.95   0.770000
96     0.96   2.400000
97     0.97   1.010000
98     0.98   5.830000
99     0.99   3.740000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  80.790001
1      0.01  80.659997
2      0.02  80.559999
3      0.03  78.340000
4      0.04  77.679998
..      ...        ...
95     0.95   0.550000
96     0.96   1.240000
97     0.97   2.230000
98     0.98   0.490000
99     0.99   0.410000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  80.790001
1      0.01  80.570000
2      0.02  79.979998
3      0.03  78.270000
4      0.04  77.149999
..      ...        ...
95     0.95   3.820000
96     0.96   0.900000
97     0.97   1.050000
98     0.98   0.380000
99     0.99   1.320000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  80.790001
1      0.01  79.900002
2  

Testing: 100%|██████████| 157/157 [00:00<00:00, 189.90it/s]


Testing accuracy of 84.040%
    Fault %   Accuracy
0      0.00  84.039998
1      0.01  83.890003
2      0.02  83.630002
3      0.03  84.030002
4      0.04  84.009999
..      ...        ...
95     0.95   0.100000
96     0.96   0.050000
97     0.97   0.160000
98     0.98   0.070000
99     0.99   0.080000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  84.039998
1      0.01  83.800000
2      0.02  83.899999
3      0.03  83.630002
4      0.04  83.530003
..      ...        ...
95     0.95   0.200000
96     0.96   0.150000
97     0.97   0.180000
98     0.98   0.150000
99     0.99   0.110000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  84.039998
1      0.01  83.780003
2      0.02  83.969998
3      0.03  83.990002
4      0.04  83.780003
..      ...        ...
95     0.95   0.320000
96     0.96   0.050000
97     0.97   0.120000
98     0.98   0.220000
99     0.99   0.070000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  84.039998
1      0.01  83.819997
2  

Testing: 100%|██████████| 157/157 [00:03<00:00, 41.68it/s]


Testing accuracy of 82.240%
    Fault %   Accuracy
0      0.00  82.239997
1      0.01  81.910002
2      0.02  81.059998
3      0.03  78.839999
4      0.04  80.510002
..      ...        ...
95     0.95   0.770000
96     0.96   0.440000
97     0.97   6.360000
98     0.98   1.460000
99     0.99   0.250000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  82.239997
1      0.01  82.209998
2      0.02  81.770003
3      0.03  81.599998
4      0.04  81.999999
..      ...        ...
95     0.95   0.290000
96     0.96   0.770000
97     0.97   0.190000
98     0.98   1.220000
99     0.99   0.770000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  82.239997
1      0.01  82.040000
2      0.02  80.280000
3      0.03  82.190001
4      0.04  77.920002
..      ...        ...
95     0.95   1.800000
96     0.96   4.150000
97     0.97   1.790000
98     0.98   0.530000
99     0.99   0.520000

[100 rows x 2 columns]
    Fault %   Accuracy
0      0.00  82.239997
1      0.01  82.389998
2  